In [2]:
import json
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from tqdm import tqdm
import os

def read_json_file(file_path):
    """지정된 경로의 JSON 파일을 읽어 데이터를 반환합니다."""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        return data
    except FileNotFoundError:
        print(f"오류: 파일을 찾을 수 없습니다 - {file_path}")
        return None
    except json.JSONDecodeError:
        print(f"오류: JSON 디코딩 오류 - {file_path}")
        return None
    except Exception as e:
        print(f"읽는 중 오류 발생 {file_path}: {e}")
        return None

def save_results_to_json(data, file_path):
    """결과 데이터를 지정된 경로의 JSON 파일로 저장합니다."""
    try:
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=4)
        print(f"결과를 '{file_path}' 파일에 성공적으로 저장했습니다.")
    except Exception as e:
        print(f"결과 파일 저장 중 오류 발생: {e}")

In [1]:
base_dir = "/home/infidea/rebirth-hjun/KRAG_2025/Dataset"
train_file_path = "/home/infidea/rebirth-hjun/KRAG_2025/Dataset/korean_language_rag_V1.0_train.json"

In [3]:
train_json_data = read_json_file(train_file_path)

In [5]:
# 각 타입의 개수를 저장할 변수 초기화
selection_type_count = 0
correction_type_count = 0

# 데이터 리스트를 순회하며 question_type 확인
for item in train_json_data:
    question_type = item.get("input", {}).get("question_type")
    if question_type == "선택형":
        selection_type_count += 1
    elif question_type == "교정형":
        correction_type_count += 1

# 결과 출력
print(f"선택형 문항 개수: {selection_type_count}")
print(f"교정형 문항 개수: {correction_type_count}")

선택형 문항 개수: 310
교정형 문항 개수: 312


In [6]:
correction_type_question_prefixes = {}

for item in train_json_data:
    if item.get("input", {}).get("question_type") == "교정형":
        question = item.get("input", {}).get("question", "")
        if "\n" in question:
            prefix = question.split("\n")[0]
            correction_type_question_prefixes[prefix] = correction_type_question_prefixes.get(prefix, 0) + 1

print("각 타입별 개수:")
for prefix, count in correction_type_question_prefixes.items():
    print(f'"{prefix}": {count}개')

각 타입별 개수:
"다음 문장에서 어문 규범에 부합하지 않는 부분을 찾아 고치고, 그 이유를 설명하세요.": 251개
"다음 중 어문 규범에 부합하는 것을 선택하고, 그 이유를 설명하세요.": 12개
"다음 문장이 어문 규범에 부합하도록 문장 부호를 추가하고, 그 이유를 설명하세요.": 19개
"다음 문장에서 어문 규범에 부합하도록 문장 부호를 추가하시오,": 1개
"다음 문장에서 어문 규범에 부합하도록 문장 부호를 추가하고, 그 이유를 설명하세요.": 7개
"다음 문장이 어문 규범에 부합하도록 문장 부호를 고치시오.": 3개
"날짜를 표기할 때, 어떻게 표기하는 것이 적절한가?": 1개
"아라비아 숫자로 연월일을 표기하고자 할 때, 다음 문장이 어문 규범에 부합하도록 문장 부호를 추가하고, 그 이유를 설명하세요.": 1개
"다음 표현이 어문 규범에 부합하도록 문장 부호를 고치시오.": 3개
"다음 중 어문 규범에 부합하는 것을 선택하고, 그 이유를 설명하세요. (두 날짜를 같이 표시할 때)": 1개
"다음 문장에서 어문 규범에 부합하지 않은 부분을 찾아 고치고, 그 이유를 설명하세요.": 1개
"다음 문장이 어문 규범에 부합하도록 괄호 안에 들어갈 문장 부호를 추가하고, 그 이유를 설명하세요.": 3개
